In [36]:
import webbrowser
import os
import parsel
from urllib.request import urlopen
from lxml.html import fromstring, tostring
from IPython.display import IFrame
from IPython.core.display import display, HTML
from IPython.display import clear_output
from ipywidgets import widgets

class interactive(object):

    def __init__(self):
        clear_output()
        self.htmlsample = ''

    def get_page(self, url):
        html = urlopen(url).read()
        dom = fromstring(html)
        dom.make_links_absolute(url)
        return dom

    def analyze(self, url):
        dom = self.get_page(url)
        self.htmlsample = tostring(dom).decode('utf-8')

    def show(self, search_term):
        
        js_selectorgadget = """
        <script>
        function myFunc() {
        var s=document.createElement('div');
        s.innerHTML='Loading...';
        s.style.color='black';
        s.style.padding='20px';
        s.style.position='fixed';
        s.style.zIndex='9999';
        s.style.fontSize='3.0em';
        s.style.border='2px solid black';
        s.style.right='40px';s.style.top='40px';
        s.setAttribute('class','selector_gadget_loading');
        s.style.background='white';
        document.body.appendChild(s);
        s=document.createElement('script');
        s.setAttribute('type','text/javascript');
        s.setAttribute('src','http://localhost:8888/tree/parttwo/selectorgadget/public/js/selectorgadget.js');
        document.body.appendChild(s);}
        </script>
        
        <script type="text/javascript">
        myFunc();
        setTimeout(function(){
            var lst = document.getElementsByClassName("sg_ignore");
            console.log(lst);
            lst[1].value='%s';
        }, 2000);
        </script>
        """ % search_term.value
        
        self.htmlsample = self.htmlsample + js_selectorgadget
        
        newsample = self.htmlsample
        #newsample = newsample.replace('<body', '<body onload="myFunc()"')

        newpath = os.path.realpath('./modified.html')

        with open(newpath, 'w') as f:
            f.write(newsample)

        display(IFrame('./modified.html', width=900, height=1000))


website = widgets.Text(description="Website to scrape", width=200)
display(website)

search_term = widgets.Text(description="Selector", width=200)
display(search_term)

button = widgets.Button(description="Check the selector")
display(button)

def on_button_clicked(b):
    obj = interactive()
    obj.analyze(str(website.value))
    obj.show(search_term)

button.on_click(on_button_clicked)